In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def exponential_rate(qi, Di, t):
    """
    Arps exponential decline:
        q(t) = qi * exp(-Di * t)
    Di: nominal decline rate (1/time)
    """
    t = np.asarray(t, dtype=float)
    qi = np.asarray(qi, dtype=float)
    Di = np.asarray(Di, dtype=float)
    if np.any(Di < 0):
        raise ValueError("Di must be >= 0")
    return qi * np.exp(-Di * t)

def hyperbolic_rate(qi, b, Di, t):
    """
    Arps hyperbolic decline:
        q(t) = qi * ((1 - Di) ** (b * t))
    b: decline acceleration (1/time)
    Di: nominal decline rate (1/time)
    """

    t = np.asarray(t, dtype=float)
    qi = np.asarray(qi, dtype=float)
    b = np.asarray(b, dtype=float)
    Di = np.asarray(Di, dtype=float)
    if np.any(Di < 0):
        raise ValueError("Di must be >= 0")
    if np.any(b < 0):
        raise ValueError("b must be >= 0")

    # Handle b=0 smoothly (limit -> exponential)
    eps = 1e-12
    if np.isscalar(b) and abs(b) < eps:
        return exponential_rate(qi, Di, t)

    # For array b, blend where b ~ 0
    out = np.empty_like(t, dtype=float)
    mask = np.abs(b) < eps
    if np.any(mask):
        out[mask] = exponential_rate(qi, Di, t[mask])
    if np.any(~mask):
        out[~mask] = qi / np.power(1.0 + b * Di * t[~mask], 1.0 / b)
    return out

def power_law_rate(qi, m, tau, t):
    """
    Power-law decline:
        q(t) = qi / (1 + t / tau) ** m

    m: dimensionless exponent (>= 0)
    tau: characteristic time scale (> 0, same units as t)
    """
    t = np.asarray(t, dtype=float)
    qi = np.asarray(qi, dtype=float)
    m = np.asarray(m, dtype=float)
    tau = np.asarray(tau, dtype=float)

    if np.any(m < 0):
        raise ValueError("m must be >= 0")
    if np.any(tau <= 0):
        raise ValueError("tau must be > 0")

    return qi / np.power(1.0 + (t / tau), m)

def linear_rate(qi, qf, t, t_end):
    """
    Linear rate interpolation from qi at t=0 to qf at t=t_end:
        q(t) = qi + (qf - qi) * (t / t_end)
    """
    t = np.asarray(t, dtype=float)
    qi = float(qi)
    qf = float(qf)
    t_end = float(t_end)
    if t_end <= 0:
        raise ValueError("t_end must be > 0")
    return qi + (qf - qi) * (t / t_end)


def flat_rate(qi, t, t_end):
    """
    Flat rate:
        q(t) = qi
    """
    t = np.asarray(t, dtype=float)
    return np.ones_like(t) * qi

In [ ]:
# Multi-segment decline sandbox
#
# This uses `playground/decline_multiseg.py` so it’s test-covered, but we convert
# to a DataFrame here for easy notebook exploration.

from pathlib import Path
import sys

# Ensure repo root is on sys.path (so `import playground...` works from notebooks)
for p in [Path.cwd(), *Path.cwd().parents]:
    if (p / "playground").exists():
        sys.path.insert(0, str(p))
        break

from playground.decline_multiseg import (
    SegmentSpec,
    simulate_multisegment,
    multisegment_to_dataframe,
)

# Example: 3 segments on a DAILY grid (365.25 days/year)
#
# Notes:
# - Time is specified in years (segment durations + Di, tau, etc.)
# - `rate` is whatever unit you choose (commonly bbl/day)
# - `cum` is integrated in DAYS, so if rate is bbl/day then cum is bbl
segments = [
    SegmentSpec(method="Hyperbolic", duration=2.0, params={"qi": 1200.0, "b": 1.2, "Di": 1.5}),
    SegmentSpec(method="Exp", duration=3.0, params={"Di": 0.25}),
    SegmentSpec(method="Flat", duration=1.0, params={}),
]

# Change frequency to "monthly" or "yearly" to test different time steps
out = simulate_multisegment(segments, frequency="daily")
df = multisegment_to_dataframe(out)

print("=" * 80)
print("ALL REQUIRED FIELDS OVERVIEW")
print("=" * 80)
print(f"\nTotal time points: {len(df)}")
print(f"Time range: {df['t_years'].min():.2f} to {df['t_years'].max():.2f} years")
print(f"              ({df['t_days'].min():.1f} to {df['t_days'].max():.1f} days)")
print(f"\nSegments: {df['segment'].unique()}")
print(f"Methods: {df['method'].unique()}")
print(f"\nInitial rate: {df['rate'].iloc[0]:.2f}")
print(f"Final rate: {df['rate'].iloc[-1]:.2f}")
print(f"Total cumulative production: {df['cum'].iloc[-1]:.2f}")
print(f"\nFinal decline from start: {df['rate_pct_change_from_start'].iloc[-1]:.1f}%")

print("\n" + "=" * 80)
print("SAMPLE DATA (First 10 rows showing all required fields)")
print("=" * 80)

# Display key columns for inspection
display_cols = [
    "t_years",
    "segment", 
    "method",
    "rate",
    "cum",
    "rate_change",
    "rate_pct_change_step",
    "rate_pct_change_from_start",
    "secant_nominal_pct_per_year",
    "secant_effective_pct_per_year",
]
df[display_cols].head(10)

# Summary statistics by segment
print("\n" + "=" * 80)
print("SUMMARY BY SEGMENT")
print("=" * 80)
segment_summary = df.groupby('segment').agg({
    'rate': ['min', 'max', 'mean'],
    'cum': 'max',
    'secant_nominal_pct_per_year': 'mean',
    'secant_effective_pct_per_year': 'mean',
    'rate_pct_change_from_start': 'min'
}).round(2)
print(segment_summary)

# Visualizations
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Rate over time
ax = axes[0, 0]
for seg in df['segment'].unique():
    seg_data = df[df['segment'] == seg]
    method = seg_data['method'].iloc[0]
    ax.plot(seg_data['t_years'], seg_data['rate'], label=f"Seg {seg}: {method}", linewidth=2)
ax.set_xlabel("Time (years)")
ax.set_ylabel("Rate")
ax.set_title("Production Rate by Segment")
ax.legend()
ax.grid(True, alpha=0.3)

# Cumulative production
ax = axes[0, 1]
ax.plot(df['t_years'], df['cum'], linewidth=2, color='darkgreen')
ax.set_xlabel("Time (years)")
ax.set_ylabel("Cumulative Production")
ax.set_title("Cumulative Production Over Time")
ax.grid(True, alpha=0.3)

# Decline rates (nominal and effective)
ax = axes[1, 0]
for seg in df['segment'].unique():
    seg_data = df[df['segment'] == seg]
    ax.plot(seg_data['t_years'], seg_data['secant_nominal_pct_per_year'], 
            label=f"Seg {seg} Nominal", linewidth=2)
ax.set_xlabel("Time (years)")
ax.set_ylabel("Decline Rate (%/year)")
ax.set_title("Secant Nominal Decline Rate")
ax.legend()
ax.grid(True, alpha=0.3)

# Percentage change from start
ax = axes[1, 1]
ax.plot(df['t_years'], df['rate_pct_change_from_start'], linewidth=2, color='red')
ax.set_xlabel("Time (years)")
ax.set_ylabel("% Change from Start")
ax.set_title("Cumulative % Change in Rate from Initial")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n" + "=" * 80)
print("✓ All required fields verified and displayed:")
print("  • Rate and cumulative production")
print("  • Current segment (1, 2, 3...)")
print("  • Calculation method (Exp, Hyperbolic, Harmonic, Linear, Flat, PowerLaw)")
print("  • Secant effective (%/year) and secant nominal (Di) tracked through time")
print("  • Change in rate (absolute and % step change)")
print("  • Cumulative % change from start")
print("=" * 80)